In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from bs4 import BeautifulSoup as soup
import requests
from lxml import html
import folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path2 = "/content/drive/My Drive/California Wildfire Predictor/Data/California_Fires.csv"
CF = pd.read_csv(path2)
CF['Started'] = pd.to_datetime(CF['Started']).dt.strftime('%Y-%m-%d')
CF['Extinguished'] = pd.to_datetime(CF['Extinguished']).dt.strftime('%Y-%m-%d')
CF.head()

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
1,30274.0,False,USFS Angeles National Forest/Los Angeles Count...,NaN,2013,True,/incidents/2013/5/30/powerhouse-fire/,NaN,NaN,Los Angeles,19,NaN,NaN,NaN,2013-06-08,NaN,False,True,NaN,NaN,NaN,34.585595,Angeles National Forest,-118.423176,False,Powerhouse Fire,100.0,NaN,True,The Powerhouse Fire burned in May and June 201...,"Powerhouse Fire, May 2013, June 2013, Angeles ...",2013-05-30,Finalized,NaN,NaN,NaN,NaN,bf37805e-1cc2-4208-9972-753e47874c87,2013-06-08T18:30:00Z,NaN
2,27531.0,False,CAL FIRE Riverside Unit / San Bernardino Natio...,NaN,2013,True,/incidents/2013/7/15/mountain-fire/,NaN,NaN,Riverside,33,NaN,NaN,NaN,2013-07-30,NaN,False,True,NaN,NaN,NaN,33.709500,Hwy 243 & Hwy 74 near Mountain Center,-116.728850,False,Mountain Fire,100.0,NaN,True,The Mountain Fire burned in July 2013 off High...,"Mountain Fire, July 2013, Highway 243, Highway...",2013-07-15,Finalized,NaN,NaN,NaN,NaN,a3149fec-4d48-427c-8b2c-59e8b79d59db,2013-07-30T18:00:00Z,NaN
3,27440.0,False,Tahoe National Forest,NaN,2013,False,/incidents/2013/8/10/american-fire/,NaN,NaN,Placer,31,NaN,NaN,NaN,2013-08-30,NaN,False,True,NaN,NaN,NaN,39.120000,"Deadwood Ridge, northeast of Foresthill",-120.650000,False,American Fire,100.0,NaN,True,The American Fire burned in August 2013 off De...,"American Fire, August 2013, Deadwood Ridge, Fo...",2013-08-10,Finalized,NaN,NaN,NaN,NaN,8213f5c7-34fa-403b-a4bc-da2ace6e6625,2013-08-30T08:00:00Z,NaN
4,24251.0,False,Ventura County Fire/CAL FIRE,NaN,2013,True,/incidents/2013/5/2/springs-fire/,Acreage has been reduced based upon more accur...,NaN,Ventura,56,47.0,8.0,117.0,2013-05-11,NaN,False,True,NaN,11.0,10.0,0.000000,Southbound Highway 101 at Camarillo Springs Ro...,0.000000,True,Springs Fire,100.0,2167.0,True,"The Springs Fire burned in May 2013, Southboun...","Springs Fire, May 2013, Highway 101, Camarillo...",2013-05-02,Finalized,6.0,10.0,NaN,NaN,46731fb8-3350-4920-bdf7-910ac0eb715c,2013-05-11T06:30:00Z,11.0


In [ ]:
path3 = "/content/drive/My Drive/California Wildfire Predictor/Data/Station_List.csv"
Station_Ids = pd.read_csv(path3)
Station_Ids = Station_Ids[["Station Number", "Name", "County", "Latitude", "Longitude", "Status"]]
Station_Ids = Station_Ids.rename(columns = {"Station Number":"Station Id","Name":"Station Name"})
Station_Ids.head()

,Station Id,Station Name,County,Latitude,Longitude,Status
0,1.0,Fresno/F.S.U. USDA,Fresno,36.814000,-119.732000,Inactive
1,2.0,Five Points,Fresno,36.336222,-120.112906,Active
2,3.0,Beach /Santa Cruz CO,Santa Cruz,36.881000,-121.793000,Inactive
3,4.0,Webb /Santa Cruz CO,Santa Cruz,36.972000,-121.726000,Inactive
4,5.0,Shafter/USDA,Kern,35.532695,-119.281862,Active


In [ ]:
path = "/content/drive/My Drive/California Wildfire Predictor/Data/merged.csv"
merged_weather = pd.read_csv(path)
merged_weather.head()

,Station Id,Date,Julian Date,Reference Eto,QC for Reference Eto,Precipitation,QC for Precipitation,Solar Radiation Average,QC for Solar Radiation Average,Average Vapor Pressure,QC for Average Vapor Pressure,Maximum Air Temperature,QC for Maximum Air Temperature,Minimum Air Temperature,QC for Minimum Air Temperature,Average Air Temperature,QC for Average Air Temperature,Maximum Relative Humidity,QC for Maximum Relative Humidity,Minimum Relative Humidity,QC for Minimum Relative Humidity,Average Relative Humidity,QC for Average Relative Humidity,Dew Point,QC for Dew Point,Average Wind Speed,QC for Average Wind Speed,Wind Run,QC for Wind Run,Average Soil Temperature,QC for Average Soil Temperature
0,2,1/1/13,1,*,205,*,47.6,*,51,*,30,*,39.8,*,6.4,*,4.7,*,0,*,92,*,55,*,0.05,*,77,*,33.1,*,113.9
1,2,1/2/13,2,*,233,*,47,*,55.4,*,26.9,*,38.6,*,6.1,*,3,*,0,*,92,*,48,*,0.05,*,76,*,31.9,*,71.9
2,2,1/3/13,3,*,231,*,46.4,*,55.6,*,25.8,*,38.6,*,6,*,3,*,0,*,93,*,48,*,0.05,*,75,*,31.4,*,71.4
3,2,1/4/13,4,*,220,*,46.1,*,55.4,*,27.1,*,40.2,*,6.5,*,3,*,0,*,94,*,52,*,0.05,*,77,*,33.5,*,72.3
4,2,1/5/13,5,*,189,*,46.3,*,55.2,*,30.3,*,43,*,7.5,*,4.2,*,0.18,*,95,*,56,*,0.04,*,80,*,37.1,*,102.3


In [ ]:
final_weather = pd.merge(left = merged_weather, right = Station_Ids, left_on = ["Station Id"], right_on = ["Station Id"], how = "left")
final_weather['Date'] = pd.to_datetime(final_weather['Date'])
final_weather.head()

,Station Id,Date,Julian Date,Reference Eto,QC for Reference Eto,Precipitation,QC for Precipitation,Solar Radiation Average,QC for Solar Radiation Average,Average Vapor Pressure,QC for Average Vapor Pressure,Maximum Air Temperature,QC for Maximum Air Temperature,Minimum Air Temperature,QC for Minimum Air Temperature,Average Air Temperature,QC for Average Air Temperature,Maximum Relative Humidity,QC for Maximum Relative Humidity,Minimum Relative Humidity,QC for Minimum Relative Humidity,Average Relative Humidity,QC for Average Relative Humidity,Dew Point,QC for Dew Point,Average Wind Speed,QC for Average Wind Speed,Wind Run,QC for Wind Run,Average Soil Temperature,QC for Average Soil Temperature,Station Name,County,Latitude,Longitude,Status
0,2,2013-01-01,1,*,205,*,47.6,*,51,*,30,*,39.8,*,6.4,*,4.7,*,0,*,92,*,55,*,0.05,*,77,*,33.1,*,113.9,Five Points,Fresno,36.336222,-120.112906,Active
1,2,2013-01-02,2,*,233,*,47,*,55.4,*,26.9,*,38.6,*,6.1,*,3,*,0,*,92,*,48,*,0.05,*,76,*,31.9,*,71.9,Five Points,Fresno,36.336222,-120.112906,Active
2,2,2013-01-03,3,*,231,*,46.4,*,55.6,*,25.8,*,38.6,*,6,*,3,*,0,*,93,*,48,*,0.05,*,75,*,31.4,*,71.4,Five Points,Fresno,36.336222,-120.112906,Active
3,2,2013-01-04,4,*,220,*,46.1,*,55.4,*,27.1,*,40.2,*,6.5,*,3,*,0,*,94,*,52,*,0.05,*,77,*,33.5,*,72.3,Five Points,Fresno,36.336222,-120.112906,Active
4,2,2013-01-05,5,*,189,*,46.3,*,55.2,*,30.3,*,43,*,7.5,*,4.2,*,0.18,*,95,*,56,*,0.04,*,80,*,37.1,*,102.3,Five Points,Fresno,36.336222,-120.112906,Active


In [ ]:
def closest_station(fire_lat, fire_lon, stations):
  stations['Distance'] = ((stations['Latitude'] - fire_lat)**2 + (stations['Longitude'] - fire_lon)**2)**.5
  stations_sort = stations.sort_values('Distance', ascending=True)
  return int(stations_sort['Station Id'].iloc[0])

In [ ]:
def days_ago(date, ago):
  split_date = date.split('-')
  final_year = split_date[0]
  year =  [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
  if final_year in ['2012', '2016', '2020']:
    year[2] = 29
  day = sum(year[:int(split_date[1])]) + int(split_date[2])
  day_ago = day - ago
  if day_ago < 0:
    final_year = str(int(final_year) - 1)
    if year[2] == 29:
      year[2] = 28
    day_ago = sum(year) + day_ago
  for i in range(len(year)):
    if day_ago > year[i+1]:
      day_ago = day_ago - year[i+1]
      continue
    else:
      return final_year + '-' + str(i+1) + '-' + str(day_ago)

In [ ]:
def find_weather_data(weather, fire, days):
  fire_started = fire['Started']
  days_before = days_ago(fire_started, days)
  dated_weather = weather[(weather['Date'] <= fire_started) & (final_weather['Date'] > days_before)]
  stations = dated_weather[['Station Id', 'Latitude', 'Longitude']]
  weather_station = closest_station(fire['Latitude'], fire['Longitude'], stations)
  return dated_weather[dated_weather['Station Id'] == weather_station]

In [ ]:
x = find_weather_data(final_weather, CF.iloc[400], 60)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
CF.iloc[400]

AcresBurned                                                            40
Active                                                              False
AdminUnit                                    CAL FIRE Lassen - Modoc Unit
AirTankers                                                            NaN
ArchiveYear                                                          2015
CalFireIncident                                                      True
CanonicalUrl                             /incidents/2015/9/9/center-fire/
ConditionStatement                                                    NaN
ControlStatement                                                      NaN
Counties                                                           Lassen
CountyIds                                                              18
CrewsInvolved                                                         NaN
Dozers                                                                NaN
Engines                               

In [ ]:
x

,Station Id,Date,Julian Date,Reference Eto,QC for Reference Eto,Precipitation,QC for Precipitation,Solar Radiation Average,QC for Solar Radiation Average,Average Vapor Pressure,QC for Average Vapor Pressure,Maximum Air Temperature,QC for Maximum Air Temperature,Minimum Air Temperature,QC for Minimum Air Temperature,Average Air Temperature,QC for Average Air Temperature,Maximum Relative Humidity,QC for Maximum Relative Humidity,Minimum Relative Humidity,QC for Minimum Relative Humidity,Average Relative Humidity,QC for Average Relative Humidity,Dew Point,QC for Dew Point,Average Wind Speed,QC for Average Wind Speed,Wind Run,QC for Wind Run,Average Soil Temperature,QC for Average Soil Temperature,Station Name,County,Latitude,Longitude,Status
111780,57,2015-07-12,193,0.21,,0,,514,,10.4,,81.6,,49.7,,65.6,,84,,28,,48,,45.5,,4.7,,113.1,,70.2,,Buntingville,Lassen,40.289799,-120.433948,Active
111781,57,2015-07-13,194,0.27,,0,,701,,9,,85.9,,45.4,,66.7,,86,,19,,40,,41.9,,4.5,,108.2,,70.5,,Buntingville,Lassen,40.289799,-120.433948,Active
111782,57,2015-07-14,195,0.26,,0,,658,,7.9,,87.9,,48.8,,69.8,,66,,15,,32,,38.5,,4.8,,115.1,,70.6,,Buntingville,Lassen,40.289799,-120.433948,Active
111783,57,2015-07-15,196,0.28,,0,,729,,8.8,,91.3,,55.8,,75,,57,,14,,30,,41.3,,4.4,,105.7,,71.9,,Buntingville,Lassen,40.289799,-120.433948,Active
111784,57,2015-07-16,197,0.3,,0,,706,,8.9,,95.1,,57.8,,77.6,,54,,14,,27,,41.4,,5.1,,121.6,,73.5,,Buntingville,Lassen,40.289799,-120.433948,Active
111785,57,2015-07-17,198,0.31,,0,,699,,7.7,,94.4,,52.5,,76.5,,58,,14,,25,,37.6,,6.3,,150.9,,73.7,,Buntingville,Lassen,40.289799,-120.433948,Active
111786,57,2015-07-18,199,0.28,,0,,714,,9.8,,89.9,,55.2,,73.8,,73,,17,,34,,44.1,,4.6,,110.1,,74.5,Y,Buntingville,Lassen,40.289799,-120.433948,Active
111787,57,2015-07-19,200,0.29,,0,,723,,8.7,,92.3,,54.6,,76.4,,61,,14,,28,,41,,4.5,,107.7,,74.9,Y,Buntingville,Lassen,40.289799,-120.433948,Active
111788,57,2015-07-20,201,0.3,,0,,669,,9.6,,95.1,,52.7,,76.1,,67,,13,,31,,43.3,,5.2,,125.6,,75.1,Y,Buntingville,Lassen,40.289799,-120.433948,Active
111789,57,2015-07-21,202,0.17,R,0.38,,417,,13,,90.8,,57.2,,70.2,,86,,21,,51,,51.4,,4.7,,112.1,,73.5,,Buntingville,Lassen,40.289799,-120.433948,Active


In [ ]:
final_weather[(final_weather['Date'] < '2015-01-01') & (final_weather['Date'] >= '2014-01-01')]['Precipitation'].value_counts()

0       43732
0.01     1670
0.02     1055
0.04      551
0.03      513
        ...  
4.03        1
1.71        1
4.23        1
2.28        1
3.26        1
Name: Precipitation, Length: 276, dtype: int64

In [ ]:
CF[CF['Counties'] == 'Tuolumne']

,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,Counties,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,Featured,Final,FuelType,Helicopters,Injuries,Latitude,Location,Longitude,MajorIncident,Name,PercentContained,PersonnelInvolved,Public,SearchDescription,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,257314.0,False,Stanislaus National Forest/Yosemite National Park,NaN,2013,True,/incidents/2013/8/17/rim-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-09-06,NaN,False,True,NaN,NaN,NaN,37.857000,3 miles east of Groveland along Hwy 120,-120.086000,False,Rim Fire,100.0,NaN,True,The Rim Fire was east of Groveland along Highw...,"Rim Fire, Stanislaus National Forest, Yosemite...",2013-08-17,Finalized,NaN,NaN,NaN,NaN,5fb18d4d-213f-4d83-a179-daaf11939e78,2013-09-06T18:30:00Z,NaN
26,1070.0,False,Stanislaus National Forest,NaN,2013,False,/incidents/2013/8/5/power-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-08-14,NaN,False,True,NaN,NaN,NaN,38.251080,"Near Beardsley Lake, northeast of Sonora",-120.027780,False,Power Fire,100.0,NaN,True,The Power Fire burned in August 2013 near Bear...,"Power Fire, August 2013, Beardsley Lake, Sonor...",2013-08-05,Finalized,NaN,NaN,NaN,NaN,1e8441c2-f426-40b4-9f4f-3de9f7e232b9,2013-08-14T08:30:00Z,NaN
87,100.0,False,CAL FIRE Tuolumne-Calaveras Unit,NaN,2013,True,/incidents/2013/6/2/murphy-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2013-06-02,NaN,False,True,NaN,NaN,NaN,37.915290,off Lime Kiln Rd,-120.333130,False,Murphy Fire,100.0,NaN,True,The Murphy Fire burned in June 2013 off Lime K...,"Murphy Fire, June 2013, Lime Kiln Road, Tuolum...",2013-06-02,Finalized,NaN,NaN,NaN,NaN,98bfbb7c-8446-4e2e-a268-7e961353723c,2013-06-02T16:55:00Z,NaN
204,487.0,False,CAL FIRE Tuolumne-Calaveras Unit,NaN,2014,True,/incidents/2014/6/4/59-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2014-06-05,NaN,False,True,NaN,NaN,NaN,37.766170,"near Bonds Flat Rd & La Grange Rd (J59), west ...",-120.450870,False,59 Fire,100.0,NaN,True,"The 59 Fire started on June 4, 2014 and burned...","59 Fire, Tuolumne County, Don Pedro Reservoir",2014-06-04,Finalized,NaN,NaN,NaN,NaN,1a0bbf7e-7f9b-4359-9c91-85dec2e2869a,2014-06-05T20:00:00Z,NaN
217,269.0,False,CAL FIRE Tuolumne-Calaveras Unit,NaN,2014,True,/incidents/2014/6/5/hetch-hetchy-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2014-06-06,NaN,False,True,NaN,NaN,NaN,37.787000,off La Grange Rd (J59) at Hetch Hetchy power l...,-120.452000,False,Hetch Hetchy Fire,100.0,NaN,True,"The Hetch Hetchy Fire started on June 5, 2014 ...","Tuolumne County, Hetch Hetchy, La Grange, Tuol...",2014-06-05,Finalized,NaN,NaN,NaN,NaN,b5d32131-cf00-4e34-9e08-b74d68a3174f,2014-06-06T09:30:00Z,NaN
247,70.0,False,CAL FIRE Tuolumne Caleveras Unit,NaN,2014,True,/incidents/2014/9/4/jackson-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2014-09-05,NaN,False,True,NaN,NaN,NaN,37.831600,Hwy 49 & Jacksonville Rd near the community of...,-120.343400,False,Jackson Fire,100.0,NaN,True,The Jackson Fire started on September 4 and bu...,"Jackson, Jackson fire, jacksonville rd, Moccas...",2014-09-04,Finalized,NaN,NaN,NaN,NaN,726710bb-32d4-46bf-9729-7f3396defcf0,2014-09-05T09:15:00Z,NaN
333,204.0,False,CAL FIRE Tuolumne-Calaveras Unit,NaN,2015,True,/incidents/2015/7/29/big-creek-fire/,Crews are continuing mop-up and fire suppressi...,NaN,Tuolumne,55,5.0,1.0,9.0,2015-08-04,NaN,False,True,NaN,NaN,NaN,37.823000,off Hwy120 and Sprague Rd near Groveland,-120.156440,True,Big Creek Fire,100.0,115.0,True,"The Big Creek Fire started on July 29, 2015 an...","Big Creek Fire, Tuolumne County, July 29, 2015...",2015-07-29,Finalized,NaN,NaN,NaN,NaN,55a1ec97-90fb-48df-8414-203b91adcd76,2015-08-04T18:00:00Z,2.0
356,112.0,False,CAL FIRE Tuolumne-Calaveras Unit,NaN,2015,True,/incidents/2015/8/15/tulloch-fire/,NaN,NaN,Tuolumne,55,NaN,NaN,NaN,2015-08-16,NaN,False,True,NaN,NaN,NaN,37.836180,"south of Hwy 108 & Tulloch 